# FFT F2PY Serial

In [4]:
%%writefile nc2cs.f90
subroutine fs(ss, ll, ts, tf, tt)
    use, intrinsic :: iso_c_binding
    include "fftw3.f03"
    double complex, intent(out) :: ss
    integer, intent(out) :: ll
    double precision, intent(out) :: ts, tf, tt
    integer, parameter :: L = 576, M = 576, N = 576
    type(C_PTR) :: plan, cdata
    complex(C_DOUBLE_COMPLEX), pointer :: data(:,:,:)
    complex(C_DOUBLE_COMPLEX) :: s
    integer :: i, j, k
    double precision :: t0, t1, t2
    
    call cpu_time(t0)    ! time measurement
           
    ! in-place transform (note dimension reversal)
    cdata = fftw_alloc_complex(int(L * M * N, C_SIZE_T))
    call c_f_pointer(cdata, data, [L, M, N])

    ! create plan for in-place forward DFT (note dimension reversal)   
    plan = fftw_plan_dft_3d(N, M, L, data, data, &
                            FFTW_FORWARD, FFTW_ESTIMATE)

    ! fills the array with complex values
    do k = 1, N
        do j = 1, M
            do i = 1, L
                data(i, j, k) = sin( real(i + j + k) )
            enddo
        enddo
    enddo
    data = dcmplx( real(data), 0 )

    call cpu_time(t1)    ! time measurement
    
    ! compute transform (as many times as desired)    
    call fftw_execute_dft(plan, data, data)
    ! checksum
    s = sum(data)

    call cpu_time(t2)    ! time measurement
               
    call fftw_destroy_plan(plan)
    call fftw_free(cdata)
     
    ! result
    ss = s * 1e-5
    ll = L
    ts = t1 - t0
    tf = t2 - t1
    tt = t2 - t0
    
end subroutine

Overwriting nc2cs.f90


In [5]:
%%bash
export FFTW_ROOT=/scratch/app/mathlibs/fftw/3.3.8_openmpi-3.1_gnu
f2py  -c nc2cs.f90  -m nc2cs  --opt='-O3'  --quiet  \
      -L$FFTW_ROOT/lib  -lfftw3  -lm  -I$FFTW_ROOT/include  \
      -DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION

{'before': '', 'this': 'use', 'after': ', intrinsic :: iso_c_binding '}


In [6]:
%%writefile nc2cs_c.py
import nc2cs
import time as tm
t2 = tm.time()    # time measurement
S, L, ts, tf, tt = nc2cs.fs()
t3 = tm.time()    # time measurement
print(f"S:{S:.0f}", end='')
print(f" | L:{L:0g}", end='')
print(f" | T1:{ts:.4f}", end='')
print(f" | TF:{tf:.4f}", end='')
print(f" | TT:{tt:.4f}", end='')
print(f" | TO:{t3-t2:.4f}")

Overwriting nc2cs_c.py


## Check

In [7]:
%%bash
module load  mathlibs/fftw/3.3.8_openmpi-3.1_gnu
time python nc2cs_c.py

S:270+0j | L:576 | T1:5.8999 | TF:13.5884 | TT:19.4883 | TO:19.4964



real	0m19.778s
user	0m19.019s
sys	0m0.709s


## Copy to /scratch

In [8]:
! cp nc2cs* /scratch${PWD#"/prj"}

## Batch script

In [9]:
%%writefile nc2cs.srm
#!/bin/bash
#SBATCH --job-name nc2cs       # Job name
#SBATCH --partition cpu_small  # Select partition
#SBATCH --ntasks=1             # Total tasks
#SBATCH --time=00:05:00        # Limit execution time
#SBATCH --exclusive            # Exclusive acccess to nodes

echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tasks per node:' $SLURM_NTASKS_PER_NODE
echo '- # of nodes in the job:' $SLURM_JOB_NUM_NODES
echo '- # of tasks:' $SLURM_NTASKS
echo '- Dir from which sbatch was invoked:' ${SLURM_SUBMIT_DIR##*/}
cd $SLURM_SUBMIT_DIR
echo -n '- List of nodes allocated to the job: '
nodeset -e $SLURM_JOB_NODELIST

# Environment
echo '-- modules ----------------------------'
cd
cd /scratch${PWD#"/prj"}/
module load mathlibs/fftw/3.3.8_openmpi-3.1_gnu
source /scratch/app/anaconda3/2020.11/etc/profile.d/conda.sh
conda activate ./env2
cd fft

# Executable
EXEC="python nc2cs_c.py"

# Start
echo '-- run --------------------------------'
echo '$ srun -n' $SLURM_NTASKS ${EXEC##*/}
echo '-- output -----------------------------'
srun -n $SLURM_NTASKS $EXEC
echo '~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

Overwriting nc2cs.srm


<hr style="height:10px;border-width:0;background-color:green">

## Submit batch script

In [10]:
%%bash
sbatch nc2cs.srm
sbatch nc2cs.srm
sbatch nc2cs.srm

Submitted batch job 1331032
Submitted batch job 1331033
Submitted batch job 1331034


In [2]:
%%bash
cat /scratch${PWD#"/prj"}/slurm-1331032.out
cat /scratch${PWD#"/prj"}/slurm-1331033.out
cat /scratch${PWD#"/prj"}/slurm-1331034.out

- Job ID: 1331032
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1449
-- modules ----------------------------
-- run --------------------------------
$ srun -n 1 python nc2cs_c.py
-- output -----------------------------
S:270+0j | L:576 | T1:7.6898 | TF:16.2453 | TT:23.9351 | TO:23.9457
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1331033
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1478
-- modules ----------------------------
-- run --------------------------------
$ srun -n 1 python nc2cs_c.py
-- output -----------------------------
S:270+0j | L:576 | T1:7.6816 | TF:16.0376 | TT:23.7192 | TO:23.7272
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1331034
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes allocate

In [11]:
! squeue -n nc2cs -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

             JOBID  PARTITION  ST  TIME NODES CPUS
           1331032  cpu_small  PD  0:00     1    1
           1331033  cpu_small  PD  0:00     1    1
           1331034  cpu_small  PD  0:00     1    1


<hr style="height:10px;border-width:0;background-color:red">